In [1]:
import pandas as pd 
import numpy as np
import boto3 
import psycopg2
from PIL import Image

In [2]:
import requests

In [3]:
import sys 
sys.path.append("../")
from backend.single_category_processor import process_category

/home/james/anaconda3/envs/matchmylook_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading function


/home/james/anaconda3/envs/matchmylook_env/lib/python3.11/site-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
/home/james/anaconda3/envs/matchmylook_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/james/anaconda3/envs/matchmylook_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Kaggle Clothing Images Dataset Pipeline

In [4]:

engine = psycopg2.connect(
    database="stealmylook",
    user="postgres",
    password="postgres",
    host="stealmylook1.ctjitqpxi57z.us-east-1.rds.amazonaws.com",
    port='5432'
)

In [5]:
engine.autocommit = True

In [6]:
cur = engine.cursor()

In [7]:
#cur.execute("CREATE EXTENSION vector;")

In [8]:
# cur.execute("""
#         drop table tops;""")

In [9]:
# cur.execute("""
#         CREATE TABLE tops (
#             id bigserial PRIMARY KEY, 
#             img_embedding vector(1363),
#             orig_link text,
#             display_name varchar(400),
#             price real,
#             gender varchar(20),
#             type varchar(100)
#         );""")

In [10]:
# cur.execute("""
#         CREATE TABLE tops (
#             id bigserial PRIMARY KEY, 
#             img_embedding vector(4096),
#             orig_link text,
#             display_name varchar(400),
#             price real,
#             type varchar(100)
#         );""")

In [11]:
# cur.execute("""
#         delete from tops;
#         """)

In [12]:
#cur.execute("CREATE INDEX ON tops USING hnsw (img_embedding vector_cosine_ops);")

In [13]:
DATA_DIR = "/mnt/c/Users/james/Downloads/kaggle_clothing/fashion-dataset/"

In [14]:
images_df = pd.read_csv(DATA_DIR+"images.csv")

In [15]:
styles_df = pd.read_csv(DATA_DIR+"styles.csv",
                        on_bad_lines="skip",
                         #error_bad_lines=False
                         )

In [16]:
images_df.head()

,filename,link
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...


In [17]:
images_df["id"] = images_df["filename"].str.replace(".jpg","").astype(int)

In [18]:
styles_df.shape

(44424, 10)

In [19]:
styles_df["id"].dtype

dtype('int64')

In [20]:
styles_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [21]:
styles_df = styles_df.merge(images_df, on="id")

In [22]:
styles_df.shape

(44424, 12)

In [23]:
styles_df["subCategory"].value_counts()

subCategory
Topwear                     15402
Shoes                        7343
Bags                         3055
Bottomwear                   2694
Watches                      2542
Innerwear                    1808
Jewellery                    1079
Eyewear                      1073
Fragrance                    1011
Sandal                        963
Wallets                       933
Flip Flops                    913
Belts                         811
Socks                         698
Lips                          527
Dress                         478
Loungewear and Nightwear      470
Saree                         427
Nails                         329
Makeup                        307
Headwear                      293
Ties                          258
Accessories                   129
Scarves                       118
Cufflinks                     108
Apparel Set                   106
Free Gifts                    104
Stoles                         90
Skin Care                      77
Sk

In [24]:
topwear = styles_df[styles_df["subCategory"]=="Topwear"]

In [25]:
topwear.shape

(15402, 12)

In [26]:
topwear["price"] = np.random.uniform(low=10, high=350, size=(len(topwear),))

/tmp/ipykernel_20739/2792358604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topwear["price"] = np.random.uniform(low=10, high=350, size=(len(topwear),))


In [27]:

#             id bigserial PRIMARY KEY, 
#             img_embedding vector(4096),
#             orig_link text,
#             display_name varchar(400),
#             price real,
#             type varchar(100)

In [28]:
topwear["productDisplayName"] = topwear["productDisplayName"].str.replace("'","")

/tmp/ipykernel_20739/3590535080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topwear["productDisplayName"] = topwear["productDisplayName"].str.replace("'","")


In [29]:
topwear.head(n=1)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link,price
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,109.118908


In [30]:
topwear["articleType"] = topwear["articleType"].str.replace("'","")


/tmp/ipykernel_20739/4065913434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topwear["articleType"] = topwear["articleType"].str.replace("'","")


In [31]:
from psycopg2.errors import UniqueViolation

In [35]:

for idx, row in topwear.head(n=100).iterrows():
    if idx % 10 == 0:
        print(f"At index {idx}")
        

    embedding = str(
        list(
            process_category(Image.open(requests.get(row["link"], stream=True).raw),"tops")
        )
    )
    topwear.at[idx, "embedding"] = embedding 
    try:
        cur.execute(f"""
            INSERT INTO tops (id,img_embedding,orig_link,display_name,price, gender, type  ) 
                    VALUES ({row['id']},'{embedding}', '{row['link']}', '{row['productDisplayName'].replace("'","")}', {row['price']}, '{row['gender']}', '{row['articleType']}')
                ; 
                    """)
    except UniqueViolation:
        print(f"Skipping {row['id']}")



At index 0
At index 30
At index 90
At index 110
At index 140


KeyboardInterrupt: 

In [36]:
# topwear_full = topwear.copy()
# topwear = topwear.iloc[:idx,:]

In [37]:
# topwear["embedding"].isnull().sum()

In [38]:

#             id bigserial PRIMARY KEY, 
#             img_embedding vector(4096),
#             orig_link text,
#             display_name varchar(400),
#             price real,
#             type varchar(100)

In [39]:
styles_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg,http://assets.myntassets.com/v1/images/style/p...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg,http://assets.myntassets.com/v1/images/style/p...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg,http://assets.myntassets.com/v1/images/style/p...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg,http://assets.myntassets.com/v1/images/style/p...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg,http://assets.myntassets.com/v1/images/style/p...


In [40]:
top_wear_write = topwear.rename(columns={
        "embedding":"img_embedding",
        "link": "orig_link",
        "productDisplayName": "display_name",
        "articleType":"type",
})

In [41]:
top_wear_write.columns.values

array(['id', 'gender', 'masterCategory', 'subCategory', 'type',
       'baseColour', 'season', 'year', 'usage', 'display_name',
       'filename', 'orig_link', 'price', 'img_embedding'], dtype=object)

In [42]:
top_wear_write = top_wear_write[["id","img_embedding","orig_link","display_name","type"]]

In [43]:
top_wear_write["price"] =  np.random.uniform(low=10, high=350, size=(len(top_wear_write),))

In [44]:
top_wear_write.head()

,id,img_embedding,orig_link,display_name,type,price
0,15970,"[0.0, 0.0, 0.0, 0.028015241, 0.029914685, 0.04...",http://assets.myntassets.com/v1/images/style/p...,Turtle Check Men Navy Blue Shirt,Shirts,120.275414
4,53759,"[0.01979104, 0.0, 0.0, 0.0, 0.0, 0.15128762, 0...",http://assets.myntassets.com/v1/images/style/p...,Puma Men Grey T-shirt,Tshirts,251.377247
5,1855,"[0.008573473, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14840...",http://assets.myntassets.com/v1/images/style/p...,Inkfruit Mens Chain Reaction T-shirt,Tshirts,16.747254
6,30805,"[0.017392058, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",http://assets.myntassets.com/v1/images/style/p...,Fabindia Men Striped Green Shirt,Shirts,326.427963
7,26960,"[0.07342745, 0.07342745, 0.0, 0.09598945, 0.09...",http://assets.myntassets.com/v1/images/style/p...,Jealous 21 Women Purple Shirt,Shirts,142.150173


In [45]:
#top_wear_write.to_sql(name="tops", con=engine, schema="stealmylook", index=False)

In [46]:
embed = topwear.at[0, "embedding"]

In [51]:
topwear.at[0, "link"]

'http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg'

In [47]:
embed

'[0.0, 0.0, 0.0, 0.028015241, 0.029914685, 0.048263725, 0.020248484, 0.01834904, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12699981, 0.12699981, 0.12699981, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07013689, 0.07013689, 0.07013689, 0.0, 0.0, 0.0, 0.031455338, 0.031455338, 0.0, 0.14540485, 0.14540485, 0.14540485, 0.054689165, 0.19757934, 0.19757934, 0.14289017, 0.0, 0.049925976, 0.049925976, 0.049925976, 0.17903484, 0.17903484, 0.032934643, 0.032934643, 0.0, 0.072525315, 0.072525315, 0.072525315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020653345, 0.020653345, 0.020653345, 0.0, 0.0, 0.089422084, 0.089422084, 0.089422084, 0.0, 0.11426213, 0.11426213, 0.07904747, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0867334, 0.09901153, 0.012278135, 0.012278135, 0.0, 0.24806525, 0.24806525, 0.24806525, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25978154, 0.25978154, 0.0, 0.0, 0.10573134, 0.10573134, 0.10573134, 0.0, 0.0, 0.0, 0.0, 0.025514802, 0.025514802, 0.025514802, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12547094, 0.1

In [61]:
res = cur.execute(f"""
    SELECT * FROM tops where gender = 'Men' ORDER BY img_embedding <-> '{embed}' LIMIT 2 OFFSET 1;
""")

In [62]:
cur.fetchall()

[(13419,
  '[0.07730843,0,0,0,0,0,0.08358083,0.08358083,0,0,0,0,0,0,0,0,0,0,0,0,0.07541144,0.07541144,0.19147195,0.19147195,0.19147195,0.009672292,0.009672292,0.011809888,0.11016326,0.11016326,0.121299595,0.03696567,0.15840381,0.14512989,0.121438146,0,0.072391145,0.072391145,0.072391145,0.03357729,0.03357729,0.03357729,0.02262308,0.02262308,0.2435765,0.22095342,0.22095342,0,0.12899001,0.12899001,0.23384398,0.10485397,0,0,0,0,0,0,0,0.09737571,0.09737571,0.26389384,0.32953298,0.32953298,0.16301487,0,0,0,0,0,0,0,0.10872521,0.10872521,0.10872521,0,0,0,0,0,0,0,0,0,0.05308269,0.05308269,0.13785905,0.08477635,0.08477635,0,0,0,0,0,0.0479469,0.0479469,0.2907375,0.24279061,0,0,0,0,0,0,0.003765339,0.09892071,0.09515537,0.09515537,0,0,0,0,0,0,0,0,0.19640543,0.23647648,0.35245845,0.11598197,0.11598197,0.025920467,0.025920467,0.21616478,0.19024432,0,0,0,0.11771461,0.11771461,0.11771461,0.16817875,0.16817875,0,0,0,0.27114934,0.27114934,0.27114934,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.14007397,0.1400739

In [104]:
res = cur.execute(f"""
    SELECT * FROM tops where id = 15970 LIMIT 5;
""")
single_img = cur.fetchall()

In [106]:
single_img[0][1]

'[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.26184884,0,0,0,0,0,0,0,0,0,0.97193104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.1229653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.59137726,0,0,0.1492258,0,0,0,0,0,2.8818562,0,0,0,0,0,0,0,0,2.043555,0,0,0,0,0,0,0,0,0.28774095,0,0,0,0,0,0,0,0,0,0,0,0,0.20764849,0,0,0,0,0,0,0,0,0.26542744,0,0,0,0,0,0.7904995,0,0,0,0,0,0,0,0,0,0,0,0.11006073,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0296192,0,0,0,0,0,0,0,0,0,0,0.8745702,0,0,0,0,0,0,2.9206114,0,0,0,0,0,0.18601906,0,0,1.4687512,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.36854905,0,0,0,0,0,0,0.24727002,2.4140925,1.3050505,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.38103148,0,0,0,0,0.72254086,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0919883,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.9658463,0,0,0,0,0.9579193,0,0,0.010222502,0,0,0,1.4561223,0,0,0,0.8773639,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.055804536,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.21498035,0,0,0,0,0,0,0,0,2.8320274,0,0,0,0,0,0,0,0.14002988,0,0,1.04786

In [108]:
embed

'[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21895203, 0.0, 0.0, 0.0, 0.07247769, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8739719, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.36647782, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34360355, 0.0, 0.0, 0.0, 0.8517641, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5496612, 0.0, 0.0, 0.0, 0.0, 0.18139893, 0.0, 0.0, 0.0, 0.37394843, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.46111822, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17040375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.30511284, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.605698, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.54793847, 0.0, 0.75451136, 0.0, 0.0

In [109]:
single_img[0][1]

'[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.26184884,0,0,0,0,0,0,0,0,0,0.97193104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.1229653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.59137726,0,0,0.1492258,0,0,0,0,0,2.8818562,0,0,0,0,0,0,0,0,2.043555,0,0,0,0,0,0,0,0,0.28774095,0,0,0,0,0,0,0,0,0,0,0,0,0.20764849,0,0,0,0,0,0,0,0,0.26542744,0,0,0,0,0,0.7904995,0,0,0,0,0,0,0,0,0,0,0,0.11006073,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0296192,0,0,0,0,0,0,0,0,0,0,0.8745702,0,0,0,0,0,0,2.9206114,0,0,0,0,0,0.18601906,0,0,1.4687512,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.36854905,0,0,0,0,0,0,0.24727002,2.4140925,1.3050505,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.38103148,0,0,0,0,0.72254086,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0919883,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.9658463,0,0,0,0,0.9579193,0,0,0.010222502,0,0,0,1.4561223,0,0,0,0.8773639,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.055804536,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.21498035,0,0,0,0,0,0,0,0,2.8320274,0,0,0,0,0,0,0,0.14002988,0,0,1.04786

In [102]:
single_img = cur.fetchall()

In [98]:
single_img[0]

IndexError: list index out of range

In [83]:
topwear.at[0, "link"]

'http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg'

In [82]:
cur.fetchall()

[(27277,
  '[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1061096,0,0,0,0,0.6530767,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.5957654,0,0,0,0,0,0,0,0,0,0,0.1027926,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.87426054,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4754039,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.59382993,0,0,0,0.42933524,0,0,0.38558543,0,0,0,0.3770231,1.520209,0,0,0,0,0,0,0,0,0,0,0,0,0,2.576005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.9690956,0,0,0,0,0,0,0,0,0,0,0,0,0.653101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.3744576,0,0,0,0,0,0,0,0,0,0.69771177,0,0.35378635,0,0,0,0,0,0.81032014,0,0,0,0,0,0,0,0.06257534,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.47159493,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.012853712,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.2515937,0,0,0,0,0,0,0,1.53127